# EDA

## I. Clean data

### Import libraries 


In [22]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


In [23]:
df = pd.read_csv('heart.csv')

In [24]:
df.head(5)


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [25]:
df.shape


(918, 12)

In [26]:
df.isna().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

In [27]:
df.duplicated().sum()

0

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [29]:
df.nunique()

Age                50
Sex                 2
ChestPainType       4
RestingBP          67
Cholesterol       222
FastingBS           2
RestingECG          3
MaxHR             119
ExerciseAngina      2
Oldpeak            53
ST_Slope            3
HeartDisease        2
dtype: int64

### Check value 0 because it could be missing value

In [33]:
# Chỉ xét các cột số
numeric_cols = ["Age","RestingBP", "Cholesterol", "MaxHR"]

# Đếm số lượng giá trị 0 trong mỗi cột
zero_counts = (df[numeric_cols] == 0).sum()

print("Số lượng giá trị 0 trong từng cột:")
print(zero_counts)


Số lượng giá trị 0 trong từng cột:
Age              0
RestingBP        1
Cholesterol    172
MaxHR            0
dtype: int64


#### have a lots of 0 values in Cholesterol 

In [50]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

# Đọc file
df = pd.read_csv("heart.csv")

# --- Xử lý RestingBP ---
# Nếu RestingBP = 0 thì thay bằng median (tính trên các giá trị > 0)
median_bp = df.loc[df["RestingBP"] > 0, "RestingBP"].median()
df["RestingBP"] = df["RestingBP"].replace(0, median_bp)

print(f"Fill RestingBP=0 as median = {median_bp}")

# --- Xử lý Cholesterol ---
# Cholesterol = 0 coi như missing
df["Cholesterol"] = df["Cholesterol"].replace(0, np.nan)

# Encode các biến phân loại
categorical_cols = ["Sex", "ChestPainType", "RestingECG", "ExerciseAngina", "ST_Slope"]
df_encoded = df.copy()
for col in categorical_cols:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col])

# Áp dụng KNN Imputer (chỉ còn Cholesterol là missing)
imputer = KNNImputer(n_neighbors=5)
df_imputed = pd.DataFrame(imputer.fit_transform(df_encoded), columns=df_encoded.columns)

# Làm tròn lại những cột nguyên
df_imputed["Age"] = df_imputed["Age"].round().astype(int)
df_imputed["RestingBP"] = df_imputed["RestingBP"].round().astype(int)
df_imputed["MaxHR"] = df_imputed["MaxHR"].round().astype(int)
df_imputed["FastingBS"] = df_imputed["FastingBS"].round().astype(int)
df_imputed["HeartDisease"] = df_imputed["HeartDisease"].round().astype(int)

# Xuất ra file CSV mới
df_imputed.to_csv("heart_final.csv", index=False)

print("✅ Dataset: heart_final.csv")


Fill RestingBP=0 as median = 130.0
✅ Dataset: heart_final.csv


### Check again


In [51]:
df = pd.read_csv("heart_final.csv")
# Chỉ xét các cột số
numeric_cols = ["Age","RestingBP", "Cholesterol", "MaxHR"]

# Đếm số lượng giá trị 0 trong mỗi cột
zero_counts = (df[numeric_cols] == 0).sum()

print("Số lượng giá trị 0 trong từng cột:")
print(zero_counts)


Số lượng giá trị 0 trong từng cột:
Age            0
RestingBP      0
Cholesterol    0
MaxHR          0
dtype: int64


## Ve bieu do